In [34]:
print("all ok")

all ok


In [35]:
from langchain_groq import ChatGroq

In [36]:
from dotenv import load_dotenv

In [37]:
load_dotenv()

True

In [38]:
#model = ChatGroq(model="wen/qwen3-32b")

In [39]:
#model.invoke("What is the capital of France?").context

In [40]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [41]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [43]:
#embedding_model.embed_query("What is the capital of France?")